In [1]:
import catboost
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error
from copy import deepcopy
sns.set()

In [2]:
X_train = pd.read_csv('lt_train.csv')
X_test = pd.read_csv('lt_test.csv')

In [3]:
X_train.describe()

,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,id
count,402902.000000,402902.000000,249593.000000,3.402342e+06,3.402342e+06,2.182591e+06,2.147736e+06,3.402342e+06,3.402342e+06,3.404846e+06
mean,866.363049,68773.279194,0.381047,4.085950e-01,1.626821e-03,2.256149e+03,1.488290e+03,1.222108e-01,7.714183e-01,1.707302e+06
std,339.571497,27970.451505,0.690858,5.081324e-01,4.030105e-02,5.514987e+02,4.408112e+02,7.633092e-01,4.199192e-01,9.833387e+05
min,309.000000,23252.000000,0.000000,0.000000e+00,0.000000e+00,4.950000e+02,9.900000e-01,0.000000e+00,0.000000e+00,1.000000e+00
25%,557.000000,37466.000000,0.033175,0.000000e+00,0.000000e+00,1.999000e+03,1.181250e+03,0.000000e+00,1.000000e+00,8.563642e+05
50%,798.000000,76559.000000,0.090909,0.000000e+00,0.000000e+00,2.399000e+03,1.434414e+03,0.000000e+00,1.000000e+00,1.707576e+06
75%,1191.000000,88915.000000,0.466667,1.000000e+00,0.000000e+00,2.600000e+03,1.762500e+03,0.000000e+00,1.000000e+00,2.558787e+06
max,1380.000000,149517.000000,27.000000,3.000000e+00,1.000000e+00,5.699000e+03,5.449000e+03,1.100000e+01,1.000000e+00,3.409998e+06


In [4]:
X_train['Date'] = pd.to_datetime(X_train['period_dt'], dayfirst = True)
X_train.set_index(['Date'], inplace=True)
X_train.drop('period_dt', axis=1, inplace=True)
X_train.head()

,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,id
Date,,,,,,,,,,
2019-12-16,764.0,23285.0,NaN,0.0,0.0,2199.0,1099.5,0.0,1.0,1
2019-12-30,764.0,23285.0,NaN,0.0,0.0,2199.0,1099.5,0.0,1.0,3
2019-12-09,764.0,23285.0,NaN,0.0,0.0,2199.0,1099.5,0.0,0.0,4
2019-12-02,453.0,23285.0,NaN,1.0,0.0,2199.0,1319.4,0.0,1.0,5
2019-12-02,764.0,23285.0,NaN,1.0,0.0,2199.0,1099.5,0.0,0.0,6


In [5]:
#создание новых переменных, отвечающих за временную структуру данных

data = X_train.reset_index().copy(deep = True)

data["monthday"] = data.Date.dt.day
data["month"] = data.Date.dt.month
data["year"] = data.Date.dt.year

In [6]:
data = data.assign(ln_demand = np.log(data['demand'] + 1))
data

,Date,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,id,monthday,month,year,ln_demand
0,2019-12-16,764.0,23285.0,NaN,0.0,0.0,2199.0,1099.5,0.0,1.0,1,16,12,2019,NaN
1,2019-12-30,764.0,23285.0,NaN,0.0,0.0,2199.0,1099.5,0.0,1.0,3,30,12,2019,NaN
2,2019-12-09,764.0,23285.0,NaN,0.0,0.0,2199.0,1099.5,0.0,0.0,4,9,12,2019,NaN
3,2019-12-02,453.0,23285.0,NaN,1.0,0.0,2199.0,1319.4,0.0,1.0,5,2,12,2019,NaN
4,2019-12-02,764.0,23285.0,NaN,1.0,0.0,2199.0,1099.5,0.0,0.0,6,2,12,2019,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3404841,2019-12-30,NaN,NaN,NaN,1.0,0.0,NaN,NaN,0.0,0.0,3409994,30,12,2019,NaN
3404842,2019-12-30,NaN,NaN,NaN,1.0,0.0,NaN,NaN,0.0,0.0,3409995,30,12,2019,NaN
3404843,2019-12-30,NaN,NaN,NaN,1.0,0.0,NaN,NaN,0.0,0.0,3409996,30,12,2019,NaN
3404844,2019-12-30,NaN,NaN,NaN,1.0,0.0,NaN,NaN,0.0,0.0,3409997,30,12,2019,NaN


In [7]:
#функции для создания лаговых фичей - ухудшило качество модели

def fill_missing_dates(x, date_col):
    min_date, max_date = x[date_col].min(), x[date_col].max()
    groupby_day = x.groupby(pd.PeriodIndex(x[date_col], freq='D'))
    results = groupby_day.sum(min_count=1)

    idx = pd.period_range(min_date, max_date)
    results = results.reindex(idx, fill_value=np.nan)

    results.index.rename(date_col, inplace=True)

    return results


def calc_preag_fill(data, group_col, date_col, target_cols, preagg_method):
    ## calc preaggregation
    data_preag = data.groupby(group_col).agg(
        preagg_method)[target_cols].reset_index()

    ## fill missing dates
    data_preag_filled = data_preag.groupby(group_col[:-1]).apply(
        fill_missing_dates, date_col=date_col).drop(group_col[:-1],
                                                    axis=1).reset_index()

    ## return DataFrame with calculated preaggregation and filled missing dates
    return data_preag_filled


def calc_rolling(data_preag_filled, group_col, date_col, method, w):

    ## calc rolling stats
    lf_df_filled = data_preag_filled.groupby(group_col[:-1]).\
        apply(lambda x: x.set_index(date_col).rolling(window=w, min_periods=1).agg(method)).drop(group_col[:-1], axis=1)

    ## return DataFrame with rolled columns from target_vars
    return lf_df_filled


def calc_ewm(data_preag_filled, group_col, date_col, span):
    ## calc ewm stats
    lf_df_filled = data_preag_filled.groupby(group_col[:-1]).\
        apply(lambda x: x.set_index(date_col).ewm(span=span).mean()).drop(group_col[:-1], axis=1)

    ## return DataFrame with rolled columns from target_vars
    return lf_df_filled


def shift(lf_df_filled, group_col, date_col, lag):

    lf_df = lf_df_filled.groupby(
        level=group_col[:-1]).apply(lambda x: x.shift(lag)).reset_index()
    lf_df[date_col] = pd.to_datetime(lf_df[date_col].astype(str))

    ## return DataFrame with following columns: filter_col, id_cols, date_col and shifted stats
    return lf_df

from ipywidgets import IntProgress

def generate_lagged_features(
        data: pd.DataFrame,
        target_cols: list = ['ln_demand'],
        id_cols: list = ['location_id', 'product_id'],
        date_col: str = 'Date',
        lags: list = [14, 28],
        windows: list = ['28D', '56D'],
        preagg_methods: list = ['mean'],
        agg_methods: list = ['mean', 'median'],
        dynamic_filters: list = ['PROMO1_FLAG'],
        ewm_params: dict = {'PROMO1_FLAG': [14, 42]}) -> pd.DataFrame:
    

    data = data.sort_values(date_col)
    out_df = deepcopy(data)
    dates = [min(data[date_col]), max(data[date_col])]

    total = len(target_cols) * len(lags) * len(windows) * len(preagg_methods) * len(agg_methods) * len(dynamic_filters)
    progress = IntProgress(min=0, max=total)
    display(progress)

    for filter_col in dynamic_filters:
        group_col = group_col = [filter_col] + id_cols + [date_col]
        for lag in lags:
            for preagg in preagg_methods:
                data_preag_filled = calc_preag_fill(data, group_col, date_col,
                                                    target_cols, preagg)

                ## add ewm features
                for alpha in ewm_params.get(filter_col, []):
                    ewm_filled = calc_ewm(data_preag_filled, group_col,
                                          date_col, alpha)
                    ewm = shift(ewm_filled, group_col, date_col, lag)
                    new_names = {x: "{0}_lag{1}d_alpha{2}_key{3}_preag{4}_{5}_dynamic_ewm".\
                        format(x, lag, alpha, '_'.join(id_cols), preagg, filter_col) for x in target_cols}

                    out_df = pd.merge(out_df,
                                      ewm.rename(columns=new_names),
                                      how='left',
                                      on=group_col)

                for w in windows:
                    for method in agg_methods:
                        rolling_filled = calc_rolling(data_preag_filled,
                                                      group_col, date_col,
                                                      method, w)

                        ## lf_df - DataFrame with following columns: filter_col, id_cols, date_col, shifted rolling stats
                        rolling = shift(rolling_filled, group_col, date_col,
                                        lag)

                        method_name = method.__name__ if type(
                            method) != str else method

                        new_names = {x: "{0}_lag{1}d_w{2}_key{3}_preag{4}_ag{5}_{6}_dynamic_rolling".\
                                     format(x, lag, w, '_'.join(id_cols), preagg, method_name, filter_col) for x in target_cols}

                        out_df = pd.merge(out_df,
                                          rolling.rename(columns=new_names),
                                          how='left',
                                          on=group_col)
                        progress.value += 1

    return out_df

In [ ]:
#target_cols = ['ln_demand']
#id_cols = ['location_id', 'product_id']
#date_col = 'Date'
#built_in_funcs = [pd.Series.kurtosis, pd.Series.skew]


#data_lagged_features = generate_lagged_features(data 
#                    , target_cols = target_cols
#                    , id_cols = id_cols
#                    , date_col = date_col
#                    , lags = [14, 28]
#                    , windows = ['28D', '56D']
#                    , preagg_methods = ['mean'] # ['mean', 'count']
#                    , agg_methods = ['mean', 'median']
#                    , dynamic_filters = ['PROMO1_FLAG']
#                    , ewm_params={'PROMO1_FLAG': [14, 28]}
#                    )

In [9]:
data.drop('Date', axis=1, inplace=True)

In [10]:
#масштабирование признаков и чистка данных от нанов в целевой переменной для трейна

data['demand'] = data['demand'].fillna(-10)
data = data.fillna(0)
sc = MinMaxScaler()
sc.fit(data.iloc[:, 3:9])
data.iloc[:, 3:9] = sc.transform(data.iloc[:, 3:9])
le = LabelEncoder()
le.fit(data['location_id'])
data['location_id'] = le.transform(data['location_id'])
le.fit(data['product_id'])
data['product_id'] = le.transform(data['product_id'])
data_test = data
data = data.loc[data['demand'] > -1]

In [11]:
#достаем айдишники только новых пар товар магазин (берем первые 10 недель появления товара в магазине)

data_new_id = []

for i in tqdm(range(1, 42)):
    data_0 = data[(data['location_id'] == i)]
    news = data_0['product_id'].tolist()
    for j in range(1, len(news)):
        if j in news:
            data_1 = data_0[(data_0['product_id'] == j)]
            if data_1.shape[0] < 10:
                data_new_id.extend(data_1['id'].tolist())
            else:
                data_1 = data_1.loc[0:10, :]
                data_new_id.extend(data_1['id'].tolist())

100%|██████████| 41/41 [00:45<00:00,  1.10s/it]


In [12]:
len(data_new_id)

50570

In [13]:
data = data[data['id'].isin(data_new_id)]
data

,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,id,monthday,month,year,ln_demand
5,3,6,0.002740,0.333333,0.0,0.368310,0.327427,0.0,1.0,7,2,1,2017,0.002736
6,3,6,1.000000,0.333333,0.0,0.385857,0.302670,0.0,1.0,8,27,2,2017,0.693147
7,3,6,1.016438,0.000000,0.0,0.385857,0.302670,0.0,1.0,9,22,5,2017,0.701333
8,3,6,0.038356,0.000000,0.0,0.385857,0.302670,0.0,1.0,10,29,5,2017,0.037639
9,3,6,0.038356,0.000000,0.0,0.385857,0.302670,0.0,1.0,11,5,6,2017,0.037639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402146,40,2541,1.190476,0.000000,0.0,0.736796,0.632993,0.0,0.0,407299,22,10,2018,0.784119
402147,40,2541,0.285714,0.333333,0.0,0.736796,0.577950,0.0,0.0,407300,29,10,2018,0.251314
402148,40,2541,0.190476,0.333333,0.0,0.736796,0.577950,0.0,0.0,407301,12,11,2018,0.174353
402149,40,2541,0.285714,0.333333,0.0,0.736796,0.577950,0.0,0.0,407302,19,11,2018,0.251314


In [15]:
test_id_list = X_test['id'].tolist()

In [44]:
#функция для проведения кросс-валидации при обучении модели (для катбуста)

def CV(X_train, y_train, number_folds, model, metrics):

    k = int(np.floor(float(X_train.shape[0]) / number_folds))

    errors = np.zeros(number_folds-1)
 
    for i in range(2, number_folds + 1):
        print('')
        split = float(i-1)/i

        X = X_train[:(k*i)]
        y = y_train[:(k*i)]

        index = int(np.floor(X.shape[0] * split))
       
        X_trainFolds = X[:index]        
        y_trainFolds = y[:index]

        X_testFold = X[(index + 1):]
        y_testFold = y[(index + 1):]

        model.fit(X_trainFolds, y_trainFolds, eval_set=[(X_testFold, y_testFold)])
        errors[i-2] = metrics(model.predict(X_testFold), y_testFold)
    
    return errors.mean()

In [ ]:
#функция кросс-валидации именно для лгбм (добавление параметра kwargs, который в катбусте используется уже хороший по умолчанию)

def CV(X_train, y_train, number_folds, model, metrics, kwargs={}):

    k = int(np.floor(float(X_train.shape[0]) / number_folds))

    errors = np.zeros(number_folds-1)
 
    for i in range(2, number_folds + 1):
        print('')
        split = float(i-1)/i

        X = X_train[:(k*i)]
        y = y_train[:(k*i)]

        index = int(np.floor(X.shape[0] * split))
       
        X_trainFolds = X[:index]        
        y_trainFolds = y[:index]

        X_testFold = X[(index + 1):]
        y_testFold = y[(index + 1):]

        model.fit(X_trainFolds, y_trainFolds, **kwargs, eval_set=[(X_testFold, y_testFold)])
        errors[i-2] = metrics(model.predict(X_testFold), y_testFold)
    
    return errors.mean()

In [17]:
data.drop('demand', axis=1, inplace=True)

In [18]:
X_train_1 = data.drop(['id','ln_demand'], axis=1) 
y_train_1 = data.ln_demand

In [ ]:
#обучаем лгбм - хуже чем катбуст

#lgbm = lgb.LGBMRegressor(
#                        n_estimators=350,
#                        learning_rate=0.1,
#                        feature_fraction=0.7,
#                        subsample=0.4,
#                        num_leaves=40)    
#
#kwargs={'early_stopping_rounds':10,'verbose':2}
#CV(X_train_1, y_train_1, 15, lgbm, mean_absolute_error, kwargs)

In [49]:
#обучаем катбуст

from catboost import CatBoostRegressor

ctb = CatBoostRegressor(loss_function = 'MAE')

CV(X_train_1, y_train_1, 10, ctb, mean_absolute_error)


0:	learn: 0.3340445	test: 0.3451627	best: 0.3451627 (0)	total: 3.33ms	remaining: 3.33s
1:	learn: 0.3293451	test: 0.3407472	best: 0.3407472 (1)	total: 5.95ms	remaining: 2.97s
2:	learn: 0.3247121	test: 0.3363159	best: 0.3363159 (2)	total: 8.9ms	remaining: 2.96s
3:	learn: 0.3209347	test: 0.3329810	best: 0.3329810 (3)	total: 11.5ms	remaining: 2.86s
4:	learn: 0.3168571	test: 0.3295216	best: 0.3295216 (4)	total: 14.2ms	remaining: 2.82s
5:	learn: 0.3130567	test: 0.3258484	best: 0.3258484 (5)	total: 16.6ms	remaining: 2.75s
6:	learn: 0.3095036	test: 0.3223124	best: 0.3223124 (6)	total: 19.2ms	remaining: 2.72s
7:	learn: 0.3061533	test: 0.3196881	best: 0.3196881 (7)	total: 21.7ms	remaining: 2.69s
8:	learn: 0.3031482	test: 0.3171933	best: 0.3171933 (8)	total: 24.3ms	remaining: 2.67s
9:	learn: 0.3000698	test: 0.3140294	best: 0.3140294 (9)	total: 27ms	remaining: 2.68s
10:	learn: 0.2967830	test: 0.3109313	best: 0.3109313 (10)	total: 29.6ms	remaining: 2.66s
11:	learn: 0.2937394	test: 0.3079271	best: 

127:	learn: 0.2033377	test: 0.2253054	best: 0.2253054 (127)	total: 311ms	remaining: 2.12s
128:	learn: 0.2031763	test: 0.2252473	best: 0.2252473 (128)	total: 314ms	remaining: 2.12s
129:	learn: 0.2030717	test: 0.2251656	best: 0.2251656 (129)	total: 316ms	remaining: 2.11s
130:	learn: 0.2029338	test: 0.2250418	best: 0.2250418 (130)	total: 318ms	remaining: 2.11s
131:	learn: 0.2027530	test: 0.2250198	best: 0.2250198 (131)	total: 321ms	remaining: 2.11s
132:	learn: 0.2026250	test: 0.2250746	best: 0.2250198 (131)	total: 323ms	remaining: 2.11s
133:	learn: 0.2024895	test: 0.2249286	best: 0.2249286 (133)	total: 326ms	remaining: 2.11s
134:	learn: 0.2023975	test: 0.2249029	best: 0.2249029 (134)	total: 329ms	remaining: 2.11s
135:	learn: 0.2022463	test: 0.2248107	best: 0.2248107 (135)	total: 332ms	remaining: 2.11s
136:	learn: 0.2020766	test: 0.2247970	best: 0.2247970 (136)	total: 335ms	remaining: 2.11s
137:	learn: 0.2019526	test: 0.2246289	best: 0.2246289 (137)	total: 338ms	remaining: 2.11s
138:	learn

265:	learn: 0.1934083	test: 0.2227337	best: 0.2220860 (225)	total: 647ms	remaining: 1.78s
266:	learn: 0.1933801	test: 0.2227345	best: 0.2220860 (225)	total: 649ms	remaining: 1.78s
267:	learn: 0.1932532	test: 0.2225528	best: 0.2220860 (225)	total: 652ms	remaining: 1.78s
268:	learn: 0.1931878	test: 0.2227139	best: 0.2220860 (225)	total: 654ms	remaining: 1.78s
269:	learn: 0.1931544	test: 0.2226832	best: 0.2220860 (225)	total: 657ms	remaining: 1.78s
270:	learn: 0.1931052	test: 0.2226813	best: 0.2220860 (225)	total: 659ms	remaining: 1.77s
271:	learn: 0.1930531	test: 0.2226789	best: 0.2220860 (225)	total: 662ms	remaining: 1.77s
272:	learn: 0.1929647	test: 0.2227056	best: 0.2220860 (225)	total: 665ms	remaining: 1.77s
273:	learn: 0.1929468	test: 0.2226804	best: 0.2220860 (225)	total: 667ms	remaining: 1.77s
274:	learn: 0.1928538	test: 0.2226744	best: 0.2220860 (225)	total: 670ms	remaining: 1.76s
275:	learn: 0.1928046	test: 0.2227443	best: 0.2220860 (225)	total: 672ms	remaining: 1.76s
276:	learn

6:	learn: 0.3132419	test: 0.3348670	best: 0.3348670 (6)	total: 23.3ms	remaining: 3.3s
7:	learn: 0.3094760	test: 0.3307059	best: 0.3307059 (7)	total: 26.7ms	remaining: 3.31s
8:	learn: 0.3061634	test: 0.3278055	best: 0.3278055 (8)	total: 29.9ms	remaining: 3.29s
9:	learn: 0.3029296	test: 0.3247681	best: 0.3247681 (9)	total: 33ms	remaining: 3.26s
10:	learn: 0.2999523	test: 0.3218612	best: 0.3218612 (10)	total: 36.2ms	remaining: 3.26s
11:	learn: 0.2968901	test: 0.3189914	best: 0.3189914 (11)	total: 39.8ms	remaining: 3.27s
12:	learn: 0.2939584	test: 0.3162128	best: 0.3162128 (12)	total: 43.2ms	remaining: 3.28s
13:	learn: 0.2911665	test: 0.3129745	best: 0.3129745 (13)	total: 46.5ms	remaining: 3.27s
14:	learn: 0.2881719	test: 0.3095727	best: 0.3095727 (14)	total: 49.6ms	remaining: 3.26s
15:	learn: 0.2854273	test: 0.3070380	best: 0.3070380 (15)	total: 52.6ms	remaining: 3.23s
16:	learn: 0.2827798	test: 0.3046091	best: 0.3046091 (16)	total: 55.9ms	remaining: 3.23s
17:	learn: 0.2802593	test: 0.302

109:	learn: 0.2029470	test: 0.2239084	best: 0.2239084 (109)	total: 356ms	remaining: 2.88s
110:	learn: 0.2027549	test: 0.2237654	best: 0.2237654 (110)	total: 359ms	remaining: 2.88s
111:	learn: 0.2025451	test: 0.2236238	best: 0.2236238 (111)	total: 363ms	remaining: 2.88s
112:	learn: 0.2023103	test: 0.2233116	best: 0.2233116 (112)	total: 366ms	remaining: 2.88s
113:	learn: 0.2020491	test: 0.2229952	best: 0.2229952 (113)	total: 370ms	remaining: 2.88s
114:	learn: 0.2018169	test: 0.2228689	best: 0.2228689 (114)	total: 375ms	remaining: 2.88s
115:	learn: 0.2017038	test: 0.2227647	best: 0.2227647 (115)	total: 378ms	remaining: 2.88s
116:	learn: 0.2014908	test: 0.2227037	best: 0.2227037 (116)	total: 382ms	remaining: 2.89s
117:	learn: 0.2012708	test: 0.2225100	best: 0.2225100 (117)	total: 386ms	remaining: 2.89s
118:	learn: 0.2010598	test: 0.2222145	best: 0.2222145 (118)	total: 390ms	remaining: 2.88s
119:	learn: 0.2009070	test: 0.2220213	best: 0.2220213 (119)	total: 393ms	remaining: 2.88s
120:	learn

...

674:	learn: 0.1971435	test: 0.2300548	best: 0.2300548 (674)	total: 5.2s	remaining: 2.5s
675:	learn: 0.1971172	test: 0.2299493	best: 0.2299493 (675)	total: 5.2s	remaining: 2.49s
676:	learn: 0.1971130	test: 0.2299435	best: 0.2299435 (676)	total: 5.21s	remaining: 2.49s
677:	learn: 0.1971087	test: 0.2299385	best: 0.2299385 (677)	total: 5.22s	remaining: 2.48s
678:	learn: 0.1971051	test: 0.2299349	best: 0.2299349 (678)	total: 5.23s	remaining: 2.47s
679:	learn: 0.1970780	test: 0.2299258	best: 0.2299258 (679)	total: 5.24s	remaining: 2.46s
680:	learn: 0.1970694	test: 0.2299241	best: 0.2299241 (680)	total: 5.24s	remaining: 2.46s
681:	learn: 0.1970681	test: 0.2299253	best: 0.2299241 (680)	total: 5.25s	remaining: 2.45s
682:	learn: 0.1970619	test: 0.2299358	best: 0.2299241 (680)	total: 5.26s	remaining: 2.44s
683:	learn: 0.1970518	test: 0.2299331	best: 0.2299241 (680)	total: 5.26s	remaining: 2.43s
684:	learn: 0.1970454	test: 0.2299428	best: 0.2299241 (680)	total: 5.27s	remaining: 2.42s
685:	learn: 0

772:	learn: 0.1958635	test: 0.2294788	best: 0.2294788 (772)	total: 5.89s	remaining: 1.73s
773:	learn: 0.1958512	test: 0.2294710	best: 0.2294710 (773)	total: 5.9s	remaining: 1.72s
774:	learn: 0.1958385	test: 0.2294529	best: 0.2294529 (774)	total: 5.91s	remaining: 1.72s
775:	learn: 0.1958371	test: 0.2294512	best: 0.2294512 (775)	total: 5.92s	remaining: 1.71s
776:	learn: 0.1958344	test: 0.2294484	best: 0.2294484 (776)	total: 5.92s	remaining: 1.7s
777:	learn: 0.1958285	test: 0.2294479	best: 0.2294479 (777)	total: 5.93s	remaining: 1.69s
778:	learn: 0.1958165	test: 0.2294274	best: 0.2294274 (778)	total: 5.94s	remaining: 1.68s
779:	learn: 0.1958131	test: 0.2294243	best: 0.2294243 (779)	total: 5.95s	remaining: 1.68s
780:	learn: 0.1957722	test: 0.2294185	best: 0.2294185 (780)	total: 5.95s	remaining: 1.67s
781:	learn: 0.1957644	test: 0.2294128	best: 0.2294128 (781)	total: 5.96s	remaining: 1.66s
782:	learn: 0.1957624	test: 0.2294096	best: 0.2294096 (782)	total: 5.97s	remaining: 1.65s
783:	learn: 

872:	learn: 0.1946975	test: 0.2287788	best: 0.2287788 (872)	total: 6.6s	remaining: 960ms
873:	learn: 0.1946973	test: 0.2287786	best: 0.2287786 (873)	total: 6.61s	remaining: 952ms
874:	learn: 0.1946870	test: 0.2287673	best: 0.2287673 (874)	total: 6.61s	remaining: 945ms
875:	learn: 0.1946857	test: 0.2287672	best: 0.2287672 (875)	total: 6.62s	remaining: 937ms
876:	learn: 0.1946705	test: 0.2287623	best: 0.2287623 (876)	total: 6.63s	remaining: 930ms
877:	learn: 0.1946490	test: 0.2287514	best: 0.2287514 (877)	total: 6.64s	remaining: 922ms
878:	learn: 0.1946473	test: 0.2287480	best: 0.2287480 (878)	total: 6.64s	remaining: 915ms
879:	learn: 0.1946379	test: 0.2287455	best: 0.2287455 (879)	total: 6.65s	remaining: 907ms
880:	learn: 0.1946159	test: 0.2287305	best: 0.2287305 (880)	total: 6.66s	remaining: 899ms
881:	learn: 0.1945937	test: 0.2287146	best: 0.2287146 (881)	total: 6.67s	remaining: 892ms
882:	learn: 0.1945737	test: 0.2286998	best: 0.2286998 (882)	total: 6.67s	remaining: 884ms
883:	learn:

971:	learn: 0.1937514	test: 0.2281393	best: 0.2281393 (971)	total: 7.29s	remaining: 210ms
972:	learn: 0.1937407	test: 0.2281407	best: 0.2281393 (971)	total: 7.3s	remaining: 203ms
973:	learn: 0.1937404	test: 0.2281403	best: 0.2281393 (971)	total: 7.31s	remaining: 195ms
974:	learn: 0.1937365	test: 0.2281270	best: 0.2281270 (974)	total: 7.32s	remaining: 188ms
975:	learn: 0.1937036	test: 0.2281295	best: 0.2281270 (974)	total: 7.33s	remaining: 180ms
976:	learn: 0.1937016	test: 0.2281272	best: 0.2281270 (974)	total: 7.33s	remaining: 173ms
977:	learn: 0.1937001	test: 0.2281254	best: 0.2281254 (977)	total: 7.34s	remaining: 165ms
978:	learn: 0.1936994	test: 0.2281250	best: 0.2281250 (978)	total: 7.35s	remaining: 158ms
979:	learn: 0.1936978	test: 0.2281279	best: 0.2281250 (978)	total: 7.36s	remaining: 150ms
980:	learn: 0.1936949	test: 0.2281234	best: 0.2281234 (980)	total: 7.36s	remaining: 143ms
981:	learn: 0.1936906	test: 0.2281219	best: 0.2281219 (981)	total: 7.37s	remaining: 135ms
982:	learn:

0.2125164998402531

In [ ]:
#обучение катбуста для каждого отдельного магазина - ухудшило качество

#for i in tqdm(range(2, 41)):
#    data1 = data[(data['location_id'] == i)]
#    data_test1 = data_test[(data_test['location_id'] == i)]
#    if data_test1.shape[0] > 0:
#        CV(data1.drop(['id','ln_demand', 'location_id'], axis=1),
#           data1.ln_demand,
#           10, ctb, mean_absolute_error)
#        final_pred = ctb.predict(data_test1.drop(['id', 'demand', 'ln_demand', 'location_id'], axis=1))
#        sub1 = pd.DataFrame({'id': data_test1['id'].tolist(), 'demand': final_pred})
#        sub = pd.concat([sub, sub1], ignore_index=False)

In [ ]:
#обучение катбуста для каждого отдельного товара - ухудшило качество

#for i in tqdm(range(1, 2823)):
#    data1 = data[(data['product_id'] == i)]
#    data_test1 = data_test[(data_test['product_id'] == i)]
#    if data_test1.shape[0] > 0:
#        CV(data1.drop(['id','ln_demand', 'product_id'], axis=1),
#           data1.ln_demand,
#           10, ctb, mean_absolute_error)
#        final_pred = ctb.predict(data_test1.drop(['id', 'demand', 'ln_demand', 'product_id'], axis=1))
#        sub1 = pd.DataFrame({'id': data_test1['id'].tolist(), 'demand': final_pred})
#        sub = pd.concat([sub, sub1], ignore_index=False)

In [50]:
data_test = data_test[data_test['id'].isin(test_id_list)]

In [51]:
data_test.describe()

,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,id,monthday,month,year,ln_demand
count,10169.000000,10169.000000,10169.0,10169.000000,10169.0,10169.000000,10169.000000,10169.000000,10169.000000,10169.000000,10169.000000,10169.000000,10169.0,10169.0
mean,24.399646,2060.097945,-10.0,0.149703,0.0,0.241706,0.146276,0.013597,0.888976,243095.325991,17.603599,6.683646,2019.0,0.0
std,12.223930,654.938097,0.0,0.170552,0.0,0.231293,0.160946,0.077478,0.314177,79756.245214,9.108064,3.280118,0.0,0.0
min,2.000000,7.000000,-10.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,601.000000,1.000000,1.000000,2019.0,0.0
25%,15.000000,1613.000000,-10.0,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,213092.000000,11.000000,4.000000,2019.0,0.0
50%,25.000000,2359.000000,-10.0,0.000000,0.0,0.289349,0.121068,0.000000,1.000000,260643.000000,19.000000,6.000000,2019.0,0.0
75%,36.000000,2556.000000,-10.0,0.333333,0.0,0.473241,0.269664,0.000000,1.000000,268235.000000,26.000000,9.000000,2019.0,0.0
max,41.000000,2821.000000,-10.0,0.666667,0.0,0.956133,1.000000,0.818182,1.000000,408054.000000,30.000000,12.000000,2019.0,0.0


In [52]:
final_pred = ctb.predict(data_test.drop(['id','demand', 'ln_demand'], axis=1))

In [53]:
sub = pd.DataFrame({'id': test_id_list, 'demand': final_pred}) 
sub['demand'] = np.exp(sub['demand']) - 1
sub.head()

,id,demand
0,601,0.614682
1,697,0.615466
2,831,0.604010
3,875,1.063882
4,975,0.610611


In [54]:
sub.to_csv('lt_sample_submission_8.csv', index=False) # катбуст